Things to learn in this section:

✔️  Use TensorFlow 2.x.  
✔️  Build, compile and train machine learning (ML) models using TensorFlow.  
✔️  Preprocess data to get it ready for use in a model.  
✔️  Use models to predict results.  
✔️  Build sequential models with multiple layers.  
✔️  Build and train models for multi-class categorization.  
✔️  Plot loss and accuracy of a trained model.  
✔️  Identify strategies to prevent overfitting, including augmentation and dropou  
✔️  Ensure that inputs to a model are in the correct shape.  
✔️  Ensure that you can match test data to the input shape of a neural network.  
✔️  Ensure you can match output data of a neural network to specified input shape for test data.  
✔️  Use callbacks to trigger the end of training cycles.  
✔️  Use datasets from tf.data.datasets.  


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

__Use datasets from tf.data.datasets__

In [2]:
ds, info = tfds.load('mnist', split='train', shuffle_files=True, with_info=True)

In [3]:
info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [4]:
images = []
labels = []
for example in tfds.as_numpy(ds):
    image, label = example["image"], example["label"]
    images.append(image)
    labels.append(label)

images = np.array(images)
images = images/255
labels = to_categorical(np.array(labels))

- __Use callbacks to trigger the end of training cycles.__  

In [5]:
ACCURACY_THRESHOLD = 0.99

# create a callback
class stopTraining(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>ACCURACY_THRESHOLD):
            print(str('\n'+str(ACCURACY_THRESHOLD)+' accuracy level reached.'))
            self.model.stop_training=True

cb_acc_level_reached = stopTraining()

- __Building A Sequential Model__
- __Understand how to use convolutions to improve your neural network__  
- __Build, compile and train machine learning (ML) models using TensorFlow__  

In [6]:
# build
model = Sequential([
    
    Conv2D(filters=16, kernel_size = (3,3), activation=tf.nn.relu, input_shape=(28, 28 ,1)),
    MaxPooling2D(pool_size=(2,2), padding='same'),
    
    Flatten(input_shape=[28,28]),
    
    Dense(128, activation=tf.nn.relu),
    Dense(10, activation=tf.nn.softmax)
])

# compile
model.compile(
    loss='categorical_crossentropy',                                               # sparse_categorical_crossentropy can be used, without one-hot-encoding for y
    optimizer=tf.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    metrics=['accuracy']
)

# train
history = model.fit(
    images,
    labels,
    epochs=10, verbose=1,
    callbacks= [cb_acc_level_reached]
         )

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 17s 281us/sample - loss: 0.2330 - accuracy: 0.9291
Epoch 2/10
60000/60000 [==============================] - 17s 283us/sample - loss: 0.0797 - accuracy: 0.9764
Epoch 3/10
60000/60000 [==============================] - 17s 286us/sample - loss: 0.0532 - accuracy: 0.9836
Epoch 4/10
60000/60000 [==============================] - 20s 336us/sample - loss: 0.0391 - accuracy: 0.9876
Epoch 5/10
59872/60000 [============================>.] - ETA: 0s - loss: 0.0293 - accuracy: 0.9910
0.99 accuracy level reached.
60000/60000 [==============================] - 19s 310us/sample - loss: 0.0293 - accuracy: 0.9910


- __Use models to predict results__  

In [7]:
ds, info = tfds.load('mnist', split='test', shuffle_files=True, with_info=True)

In [18]:
images_test = []
labels_test = []
for example in tfds.as_numpy(ds):
    image, label = example["image"], example["label"]
    images_test.append(image)
    labels_test.append(label)

images_test = np.array(images_test)
images_test = images_test/255

In [25]:
predictions = model.predict(images_test)
predictions = np.argmax(predictions, axis=1) # one-hot-encoding to integer

In [26]:
predictions

array([2, 0, 4, ..., 8, 0, 5])

In [24]:
np.sum(np.array(labels_test) == predictions)

9842

- __Plotting Accuracy__

In [9]:
plt.plot(acc)